In [4]:
import random
import math
import numpy as np
import copy
import getopt
import psycopg2
import sys
import time
import matplotlib.pyplot as plt


In [5]:
def readInput(mode):
    global D
    global n
    #mode 0 bank, mode 1 adult
    if mode==0:
        input_path = "./bank_marketing.csv"
        input_file = open(input_path,'r')
        n = len(input_file.readlines())-1
        input_file.close()
        D = np.zeros(n)  
        i = 0
        input_file = open(input_path,'r')
        for line in input_file.readlines():
                elements = line.split(",")
                if i>0:
                    D[i-1] = int(elements[5])
                i+=1

        D.sort()
    if mode==1:
        input_path = "./adult.csv"
        input_file = open(input_path,'r')
        n = len(input_file.readlines())-1
        input_file.close()
        D = np.zeros(n)  
        i = 0
        input_file = open(input_path,'r')
        for line in input_file.readlines():
                elements = line.split(",")
                if i>0:
                    D[i-1] = int(elements[2])
                i+=1
        D.sort()


In [150]:
# def discretize(D, b):    
#     n = len(D)
#     discreteD = np.zeros(n)
#     for i in range(n):
#         discreteD[i] = int(D[i]/b)
#     return discreteD
def count(D, a, b):
    counter = 0
    n = len(D)
    for i in range(n):
        if D[i] >=a and D[i]<=b:
            counter+=1
    return counter

def clip(D, a, b):
    clipped = copy.deepcopy(D)
    clipped[clipped<a] = a
    clipped[clipped>b] = b
    return clipped

def LapNoise():
    a = random.uniform(0,1)
    b = math.log(1/(1-a))
    c = random.uniform(0,1)
    if c>0.5:
        return b
    else:
        return -b
    
def F(x):
    return 1/2+1/(4*math.pi)*(math.log(abs(2*x**2+2*math.sqrt(2)*x+2)/abs(abs(2*x**2-2*math.sqrt(2)*x+2)))+2*math.atan(math.sqrt(2)*x+1)+2*math.atan(math.sqrt(2)*x-1))
def inver_F(y):
    #find the solution of F(x)=y
    #Find between -1000000 and 1000000 because F(1000000)=1.0 in python
    if y>1/2:
        low =0.0
        high = 1000000.0
        mid = (high+low)/2
        while abs(high-low)>0.0000001:
            if F(mid)>y:
                high=mid
            elif F(mid)==y:
                return mid
            else: 
                low = mid
            mid = (high+low)/2
            
        return high
    if y==1/2:
        return 0
    
def CauchyNoise():
    a = random.uniform(0.5,1)
    b = inver_F(a)
    c = random.uniform(0,1)
    if c>0.5:
        return b
    else:
        return -b

# def finiteDomainMedian(eps, a,b, D):
#     #[a,b] denotes range
#     n = len(D)
#     u = np.zeros(n+2)#utility score
#     l = np.zeros(n+2)#score changing point
#     l[0] = a
#     for i in range(n+1):
#         if i<=int(n/2):
#             u[i] = -int(n/2)-1+i
#             l[i+1]= D[i]
#         #u[int(n/2+1)] = 0
#         #l[int(n/2+2)] = D[int(n/2)]
#         if i>int(n/2):
#             u[i] = int(n/2)+1-i
#             l[i]= D[i-1]
            
#     l[n+1] = b
#     u[n+1] = -n-1-u[0]
#     #merge to delete duplicates
#     u_merge = []
#     l_merge = []
#     for i in range(int(n/2)+1):
#         if l[i+1]==l[i]:
#             continue
#         else:
#             l_merge.append(l[i])
#             u_merge.append(u[i])

#     temp=int(n/2)+1
#     idx =0
#     for i in range(int(n/2)+1,n+1):
#         if l[i+1]==l[i]:
#             continue
#         else:
#             l_merge.append(l[i])
#             u_merge.append(u[temp])
#             if temp==int(n/2)+1:
#                 idx = len(l_merge)
#             temp = i+1
#     l_merge.append(b)
#     u_merge.append( -n-1-u[0])
#     #print(u_merge)  
# #     l = 0
# #     for i in range(len(u_merge)-1):
# #         l = max(l, u_merge[i+1]-u_merge[i])
        
#     #print("max gap"+str(l))
#     weight = []
#     for i in range(idx-1):
#         weight.append((l_merge[i+1]-l_merge[i])*math.pow(np.e, eps*u_merge[i]/2))
#     weight.append(1)
#     for i in range(idx,len(l_merge)):
#         weight.append((l_merge[i]-l_merge[i-1])*math.pow(np.e, eps*u_merge[i]/2))

#     totalWeight = sum(weight)
#     weight = weight/totalWeight
#     i = np.random.choice(list(range(len(l_merge))), p=weight)
#     if i==idx:
#         return l_merge[i]
#     if i<idx:
#         return np.random.randint(l_merge[i], l_merge[i+1],dtype=np.int64)
#     if i>idx:
#         return np.random.randint(l_merge[i-1], l_merge[i],dtype=np.int64)
    

In [179]:
def discrete(D):
    n = len(D)
    newD = np.zeros(n)
    previous = -1
    for i in range(n):
        if D[i]!=previous:
            newD[i] = n*D[i]
            #print(newD[i])
            previous = D[i]
        elif D[i]==previous:
            newD[i] = newD[i-1]+1
            
    return newD

def constructu(eps, a,b, D):
    global u
    global l
    global weight
    #[a,b] denotes range
    n = len(D)
    u = np.zeros(n+2)#utility score
    l = np.zeros(n+2)#score changing point
    l[0] = a
    for i in range(n+1):
        if i<=int(n/2):
            u[i] = -int(n/2)-1+i
            l[i+1]= D[i]
        #u[int(n/2+1)] = 0
        #l[int(n/2+2)] = D[int(n/2)]
        if i>int(n/2):
            u[i] = int(n/2)+1-i
            l[i]= D[i-1]
            
    l[n+1] = b
    u[n+1] = -n-1-u[0]
    weight = []
    for i in range(int(n/2)+1):
        weight.append((l[i+1]-l[i])*math.pow(np.e, eps*u[i]/2))
    weight.append(1)
    for i in range(int(n/2)+1,n+1):
        weight.append((l[i]-l[i-1])*math.pow(np.e, eps*u[i]/2))

    totalWeight = sum(weight)
    weight = weight/totalWeight
    #print(len(weight))
    
def EMMedian_new():
    i = np.random.choice(list(range(len(l))), p=weight)
    if i==int(n/2)+1:
        return int(l[i]/n)
    if i<int(n/2)+1:
        return int(np.random.randint(l[i], l[i+1],dtype=np.int64)/n)
    if i>int(n/2)+1:
        return int(np.random.randint(l[i-1], l[i],dtype=np.int64)/n)
    
def constructu_CI(eps, beta, N):
    global u1
    global weight1
    global u2
    global weight2
    global factor
    
    factor = int(8/eps*np.log(4*n*N/beta))
    #print("factor is "+str(factor))
    u1 = np.zeros(n+2)
    u2 = np.zeros(n+2)
    for i in range(n+2):
        if i<=int(n/2)+1:
            u1[i] = -abs(u[i]+factor)
            u2[i] = u[i]-factor
        else:
            u1[i] = u[i]-factor
            u2[i] = -abs(u[i]+factor)
            
    idx = int(n/2)+1-factor
    weight1 = []
    for i in range(idx):
        weight1.append((l[i+1]-l[i])*math.pow(np.e, eps*u1[i]/4))
    weight1.append(1)
    for i in range(idx,n+1):
        weight1.append((l[i]-l[i-1])*math.pow(np.e, eps*u1[i]/4))

    totalWeight1 = sum(weight1)
    weight1 = weight1/totalWeight1
    
    idx = int(n/2)+1+factor
    weight2 = []
    for i in range(idx):
        weight2.append((l[i+1]-l[i])*math.pow(np.e, eps*u2[i]/4))
    weight2.append(1)
    for i in range(idx,n+1):
        weight2.append((l[i]-l[i-1])*math.pow(np.e, eps*u2[i]/4))

    totalWeight2 = sum(weight2)
    weight2 = weight2/totalWeight2
    
def EMMedianCI():
    i1 = np.random.choice(list(range(len(l))), p=weight1)
    if i1==int(n/2)+1-factor:
        x1= int(l[i1]/n)
    if i1<int(n/2)+1-factor:
        x1= int(np.random.randint(l[i1], l[i1+1],dtype=np.int64)/n)
    if i1>int(n/2)+1-factor:
        x1= int(np.random.randint(l[i1-1], l[i1],dtype=np.int64)/n)
    
    i2 = np.random.choice(list(range(len(l))), p=weight2)
    if i2==int(n/2)+1+factor:
        x2= int(l[i2]/n)
    if i2<int(n/2)+1+factor:
        x2= int(np.random.randint(l[i2], l[i2+1],dtype=np.int64)/n)
    if i2>int(n/2)+1+factor:
        x2= int(np.random.randint(l[i2-1], l[i2],dtype=np.int64)/n)
    
    est = EMMedian_new()
    indicator=0
    if D[int(n/2)]<=x2 and D[int(n/2)]>=x1:
        indicator=1
        
    return x1,x2 ,indicator,est

In [154]:
# def EMMedian(eps, beta,  D):
#     n = len(D)
#     rad = 10000000
#     if mode==0:
#         left=-rad
#         est = finiteDomainMedian(eps, left,rad, D)
#     if mode==1:
#         left=0
#         est = finiteDomainMedian(eps, left,rad, D)
    
#     #define utility function
#     u = np.zeros(n+2)#utility score
#     l = np.zeros(n+2)#score changing point
#     l[0] = left
#     for i in range(n+1):
#         if i<=int(n/2):
#             u[i] = -int(n/2)-1+i
#             l[i+1]= D[i]
#         #u[int(n/2+1)] = 0
#         #l[int(n/2+2)] = D[int(n/2)]
#         if i>int(n/2):
#             u[i] = int(n/2)+1-i
#             l[i]= D[i-1]
            
#     l[n+1] = rad
#     u[n+1] = -n-1-u[0]
#     #define u1
#     factor = int(9/eps*np.log(2*(rad-left)*len(D)/beta))+1
#     u1 = np.zeros(n+2)
#     u2 = np.zeros(n+2)
#     for i in range(n+2):
#         if i<=int(n/2):
#             u1[i] = -abs(u[i]+factor)
#             u2[i] = u[i]-factor
#         else:
#             u1[i] = u[i]-factor
#             u2[i] = -abs(u[i]+factor)
          
#     u1_merge = []
#     l_merge = []
#     for i in range(int(n/2)+1-factor):
#         if l[i+1]==l[i]:
#             continue
#         else:
#             l_merge.append(l[i])
#             u1_merge.append(u1[i])

#     temp=int(n/2)+1-factor
#     idx =0
#     for i in range(int(n/2)+1-factor,n+1):
#         if l[i+1]==l[i]:
#             continue
#         else:
#             l_merge.append(l[i])
#             u1_merge.append(u1[temp])
#             if temp==int(n/2)+1-factor:
#                 idx = len(l_merge)
#             temp = i+1
            
#     l_merge.append(rad)
#     u1_merge.append( -n-1-u1[0])

#     weight1 = []
#     for i in range(idx-1):
#         weight1.append((l_merge[i+1]-l_merge[i])*math.pow(np.e, eps*u1_merge[i]/2))
#     weight1.append(1)
#     for i in range(idx,len(l_merge)):
#         weight1.append((l_merge[i]-l_merge[i-1])*math.pow(np.e, eps*u1_merge[i]/2))

#     totalWeight1 = sum(weight1)
#     weight1 = weight1/totalWeight1
#     i = np.random.choice(list(range(len(l_merge))), p=weight1)
#     if i==idx:
#         x1= l_merge[i]
#     if i<idx:
#         x1= np.random.randint(l_merge[i], l_merge[i+1])
#     if i>idx:
#         x1= np.random.randint(l_merge[i-1], l_merge[i])
        
#     u2_merge  = []  
#     for i in range(int(n/2)+1+factor):
#         if l[i+1]==l[i]:
#             continue
#         else:
#             u2_merge.append(u2[i])

#     temp=int(n/2)+1+factor
#     idx =0
#     for i in range(int(n/2)+1+factor,n+1):
#         if l[i+1]==l[i]:
#             continue
#         else:
#             u2_merge.append(u2[temp])
#             if temp==int(n/2)+1+factor:
#                 idx = len(u2_merge)
#             temp = i+1
            
#     u2_merge.append( -n-1-u2[0])
#     weight2 = []
#     for i in range(idx-1):
#         weight2.append((l_merge[i+1]-l_merge[i])*math.pow(np.e, eps*u2_merge[i]/2))
#     weight2.append(1)
#     for i in range(idx,len(l_merge)):
#         weight2.append((l_merge[i]-l_merge[i-1])*math.pow(np.e, eps*u2_merge[i]/2))

#     totalWeight2 = sum(weight2)
#     weight2 = weight2/totalWeight2
#     i = np.random.choice(list(range(len(l_merge))), p=weight2)
#     if i==idx:
#         x2= l_merge[i]
#     if i<idx:
#         x2= np.random.randint(l_merge[i], l_merge[i+1])
#     if i>idx:
#         x2= np.random.randint(l_merge[i-1], l_merge[i])    
        

#     indicator=0
#     if D[int(n/2)]<=x2 and D[int(n/2)]>=x1:
#         indicator=1
        
        
#     return x1,x2 ,indicator,est
    

In [5]:
def smoothMedian(eps, beta, D,delta=0):
    #given D, first compute S(D)
    n = len(D)
    m = int(n/2)
    rad = 10000000
    SS = 0
    if delta==0:
        alpha = eps/20
        for k in range(500):
            for t in range(k+2):
                if m+t>n-1:
                    rightx = rad
                else:
                    rightx = D[m+t]
                if m+t-k-1 <0:
                    leftx = -rad
                else:
                    leftx = D[m+t-k-1]
                temp = math.pow(math.e, -k*alpha)*(rightx-leftx)
                SS = max(SS,temp)
        #print(SS)        
        noisySS = SS*math.pow(math.e, 2*alpha/eps*LapNoise()+2*alpha/eps*math.log(2/beta))
        #print("ratio "+str(noisySS/SS))

        Q = D[int(n/2)]+20*SS/eps*CauchyNoise()
        #print(Q)
        indicator=0
        factor = 20*noisySS/eps*inver_F(1-beta/4)
        if D[int(n/2)]<=Q+ factor and D[int(n/2)]>=Q-factor:
            indicator=1

        return Q, factor,indicator
    else:
        alpha = eps/(4*math.log(1/delta))
        for k in range(500):
            for t in range(k+2):
                if m+t>n-1:
                    rightx = rad
                else:
                    rightx = D[m+t]
                if m+t-k-1 <0:
                    leftx = -rad
                else:
                    leftx = D[m+t-k-1]
                temp = math.pow(math.e, -k*alpha)*(rightx-leftx)
                SS = max(SS,temp)
        #print(SS)
        noisySS = SS*math.pow(math.e, 2*alpha/eps*LapNoise()+2*alpha/eps*math.log(2/beta))
        #print(noisySS) 
        Q = D[int(n/2)]+4*SS/eps*LapNoise()
        
        
        indicator=0
        if D[int(n/2)]<=Q+ 4*noisySS/eps*math.log(2/beta) and D[int(n/2)]>=Q-4*noisySS/eps*math.log(2/beta):
            indicator=1

        return Q, 4*noisySS/eps*math.log(2/beta),indicator
    

In [6]:
def SVTquantile(eps, beta, D, tau, rad):
    start = time.time()
    TNoise = 2/eps*LapNoise()
    T = tau
    
    for i in range(1000):
        Ti = T-4/eps*np.log(2*(i+1)**2 * np.pi**2/(3*beta)) - 2/eps*np.log(4/beta)+TNoise
        Qi = counts[i]+4/eps*LapNoise()
        if Qi>Ti:
            end = time.time()
            #print("time is "+str(end-start))
            return -rad+i
      

    if mode==0:
        start = int(rad+D[0])
        for i in range(start,2*rad+1):
            Ti = T-4/eps*np.log(2*(i+1)**2 * np.pi**2/(3*beta)) - 2/eps*np.log(4/beta)+TNoise
            Qi = counts[i]+4/eps*LapNoise()
            if Qi>Ti:
                end = time.time()
                #print("time is "+str(end-start))
                return -rad+i
    if mode==1:
        start = int(D[0])
        for i in range(start,rad+1):
            Ti = T-4/eps*np.log(2*(i+1)**2 * np.pi**2/(3*beta)) - 2/eps*np.log(4/beta)+TNoise
            Qi = counts[i]+4/eps*LapNoise()
            if Qi>Ti:
                end = time.time()
                #print("time is "+str(end-start))
                return i

def SVTmedian(eps, D, rad):
    start = time.time()
    TNoise = 2/eps*LapNoise()
    T = int(n/2)
    
    for i in range(rad+1):
        Ti = T +TNoise
        Qi = counts[i]+4/eps*LapNoise()
        if Qi>Ti:
            end = time.time()
            #print("time is "+str(end-start))
            return i
def SVTMedianCI(eps,beta, D):
    global mode
    mode=1
    n = len(D)
    rad = 10000000
    if mode==0:
        left=-rad
    if mode==1:
        left=0
    Q = SVTmedian(eps, D,  rad-left)
    
    factor = int(16/eps*math.log(2*(rad-left+1)**2 *math.pi**2/(3*beta))+8/eps*math.log(4/beta))+1
    ##Then run SVT, do not double
    ##find the n/2 +- factor quantile
    T1 = int(n/2)-1
    T2 = int(n/2)+factor
    XLeft = SVTquantile(eps/2, beta/2, D, T1, rad)
    #print(XLeft)
    XRight = SVTquantile(eps/2, beta/2, D, T2, rad)
    #print(XRight)
    indicator=0
    if D[int(n/2)]<=Q+ XRight and D[int(n/2)]>=XLeft:
        indicator=1
    
    return Q, XLeft,XRight,indicator

In [185]:
mode=0
readInput(mode)
print(D)
print(n)
newD = discrete(D)
print(newD)
print(D)


[ -8019.  -6847.  -4057. ...  81204.  98417. 102127.]
45211
[-3.62547009e+08 -3.09559717e+08 -1.83421027e+08 ...  3.67131404e+09
  4.44953099e+09  4.61726380e+09]
[ -8019.  -6847.  -4057. ...  81204.  98417. 102127.]


In [186]:
eps = [0.125, 0.25, 0.5, 1,2,4]
beta = [0.01, 0.025, 0.05, 0.1, 0.2]
len_eps = []
err_eps = []
for i in range(6):
    print("Results for EM based median, Eps = "+str(eps[i]))
    errors = []
    lengths = []
    correct_count = 0
    n = len(newD)
    rad = 10000000
    if mode==0:
        left=-rad
        constructu(eps[i], left*n,rad*n, newD)
    if mode==1:
        left=0
        constructu(eps[i], left*n,rad*n, newD)
        
    constructu_CI(eps[i],beta[3], rad)

    num_repeat=100
    for j in range(num_repeat):
        xl,xr,indi,est = EMMedianCI()
        correct_count+=indi
        errors.append(abs(est-D[int(n/2)]))
        lengths.append((xr-xl)/(2))

    print("correct rate = " +str(correct_count/num_repeat))
    avgLength = sum(lengths)/num_repeat
    errors.sort()
    errorQuantile = errors[int(num_repeat*(1-beta[3]))]
    print("Average CI length = "+str(avgLength))
    print("error quantile is "+ str(errorQuantile))
    print("CI length/ error quantile = "+str(avgLength/errorQuantile))
    len_eps.append(avgLength)
    err_eps.append(errorQuantile)
print(len_eps)
print(err_eps)


Results for EM based median, Eps = 0.125
correct rate = 1.0
Average CI length = 91.27
error quantile is 3.0
CI length/ error quantile = 30.423333333333332
Results for EM based median, Eps = 0.25
correct rate = 1.0
Average CI length = 46.59
error quantile is 1.0
CI length/ error quantile = 46.59
Results for EM based median, Eps = 0.5
correct rate = 1.0
Average CI length = 23.545
error quantile is 1.0
CI length/ error quantile = 23.545
Results for EM based median, Eps = 1
correct rate = 1.0
Average CI length = 12.075
error quantile is 1.0
CI length/ error quantile = 12.075
Results for EM based median, Eps = 2
correct rate = 1.0
Average CI length = 5.51
error quantile is 1.0
CI length/ error quantile = 5.51
Results for EM based median, Eps = 4
correct rate = 1.0
Average CI length = 3.28
error quantile is 1.0
CI length/ error quantile = 3.28
[91.27, 46.59, 23.545, 12.075, 5.51, 3.28]
[3.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [187]:
eps = [0.125, 0.25, 0.5, 1,2,4]
beta = [0.01, 0.025, 0.05, 0.1, 0.2]
len_beta = []
err_beta = []
for i in range(5):
    print("Results for EM based median, beta = "+str(beta[i]))
    errors = []
    lengths = []
    correct_count = 0
    n = len(newD)
    rad = 10000000
    if mode==0:
        left=-rad
        constructu(eps[3], left*n,rad*n, newD)
    if mode==1:
        left=0
        constructu(eps[3], left*n,rad*n, newD)
        
    constructu_CI(eps[3],beta[i], rad)
    num_repeat=100
    for j in range(num_repeat):
        xl,xr,indi,est = EMMedianCI()
        correct_count+=indi
        errors.append(abs(est-D[int(n/2)]))
        lengths.append((xr-xl)/(2))

    print("correct rate = " +str(correct_count/num_repeat))
    avgLength = sum(lengths)/num_repeat
    errors.sort()
    errorQuantile = errors[int(num_repeat*(1-beta[i]))]
    print("Average CI length = "+str(avgLength))
    print("error quantile is "+ str(errorQuantile))
    print("CI length/ error quantile = "+str(avgLength/errorQuantile))
    len_beta.append(avgLength)
    err_beta.append(errorQuantile)
print(len_beta)
print(err_beta)

Results for EM based median, beta = 0.01
correct rate = 1.0
Average CI length = 13.17
error quantile is 1.0
CI length/ error quantile = 13.17
Results for EM based median, beta = 0.025
correct rate = 1.0
Average CI length = 12.45
error quantile is 1.0
CI length/ error quantile = 12.45
Results for EM based median, beta = 0.05
correct rate = 1.0
Average CI length = 12.645
error quantile is 1.0
CI length/ error quantile = 12.645
Results for EM based median, beta = 0.1
correct rate = 1.0
Average CI length = 12.06
error quantile is 1.0
CI length/ error quantile = 12.06
Results for EM based median, beta = 0.2
correct rate = 1.0
Average CI length = 11.89
error quantile is 1.0
CI length/ error quantile = 11.89
[13.17, 12.45, 12.645, 12.06, 11.89]
[1.0, 1.0, 1.0, 1.0, 1.0]


In [28]:
eps = [0.125, 0.25, 0.5, 1,2,4]
beta = [0.01, 0.025, 0.05, 0.1, 0.2]
len_eps = []
err_eps = []
for i in range(6):
    print("Results for Smooth based median, pure DP "+str(eps[i]))
    errors = []
    lengths = []
    correct_count = 0
    num_repeat=100
    for j in range(num_repeat):
        Q,length,indi = smoothMedian(eps[i], beta[3], D,delta=0)
        correct_count+=indi
        errors.append(abs(Q-D[int(n/2)]))
        lengths.append(length)

    print("correct rate = " +str(correct_count/num_repeat))
    avgLength = sum(lengths)/num_repeat
    errors.sort()
    errorQuantile = errors[int(num_repeat*(1-beta[3]))]
    print("Average CI length = "+str(avgLength))
    print("error quantile is "+ str(errorQuantile))
    print("CI length/ error quantile = "+str(avgLength/errorQuantile))
    len_eps.append(avgLength)
    err_eps.append(errorQuantile)
print(len_eps)
print(err_eps)



Results for Smooth based median, pure DP0.125
correct rate = 1.0
Average CI length = 104500.32157916682
error quantile is 48650.598949362306
CI length/ error quantile = 2.147976054476439
Results for Smooth based median, pure DP0.25
correct rate = 0.97
Average CI length = 31361.001125639137
error quantile is 17245.829206519556
CI length/ error quantile = 1.8184687294585713
Results for Smooth based median, pure DP0.5
correct rate = 0.99
Average CI length = 9307.377131687417
error quantile is 6551.269038561266
CI length/ error quantile = 1.4206983527776817
Results for Smooth based median, pure DP1
correct rate = 0.97
Average CI length = 3034.791203584908
error quantile is 2031.979859488696
CI length/ error quantile = 1.4935144112838539
Results for Smooth based median, pure DP2
correct rate = 0.98
Average CI length = 1020.1617359274985
error quantile is 587.099356349645
CI length/ error quantile = 1.7376304792266624
Results for Smooth based median, pure DP4
correct rate = 0.98
Average CI l

In [93]:
eps = [0.125, 0.25, 0.5, 1,2,4]
beta = [0.01, 0.025, 0.05, 0.1, 0.2]
len_beta = []
err_beta = []
for i in range(5):
    print("Results for Smooth based median, pure DP beta = "+str(beta[i]))
    errors = []
    lengths = []
    correct_count = 0
    num_repeat=100
    for j in range(num_repeat):
        Q,length,indi = smoothMedian(eps[3], beta[i], D,delta=0)
        correct_count+=indi
        errors.append(abs(Q-D[int(n/2)]))
        lengths.append(length)

    print("correct rate = " +str(correct_count/num_repeat))
    avgLength = sum(lengths)/num_repeat
    errors.sort()
    errorQuantile = errors[int(num_repeat*(1-beta[i]))]
    print("Average CI length = "+str(avgLength))
    print("error quantile is "+ str(errorQuantile))
    print("CI length/ error quantile = "+str(avgLength/errorQuantile))
    len_beta.append(avgLength)
    err_beta.append(errorQuantile)
print(len_beta)
print(err_beta)

Results for Smooth based median, pure DP beta = 0.01
correct rate = 1.0
Average CI length = 8193.00714748021
error quantile is 3881.8658616715693
CI length/ error quantile = 2.110584816537741
Results for Smooth based median, pure DP beta = 0.025
correct rate = 0.99
Average CI length = 5718.378704640575
error quantile is 3237.7940319238987
CI length/ error quantile = 1.7661341790919023
Results for Smooth based median, pure DP beta = 0.05
correct rate = 0.98
Average CI length = 4286.4565613012655
error quantile is 2706.445757813868
CI length/ error quantile = 1.5837954811862371
Results for Smooth based median, pure DP beta = 0.1
correct rate = 0.99
Average CI length = 3075.5718691935635
error quantile is 1681.4147261963517
CI length/ error quantile = 1.829157209864002
Results for Smooth based median, pure DP beta = 0.2
correct rate = 0.89
Average CI length = 2210.01498664275
error quantile is 1573.215812101931
CI length/ error quantile = 1.4047754730420672
[8193.00714748021, 5718.3787046

In [98]:
print("Results for Smooth based median, approximate DP")
errors = []
lengths = []
correct_count = 0
num_repeat=100
eps=1
beta=0.1
for i in range(num_repeat):
    Q,length,indi = smoothMedian(eps, beta, D,delta=0.0000001)
    correct_count+=indi
    errors.append(abs(Q-D[int(n/2)]))
    lengths.append(length)
    
print("correct rate = " +str(correct_count/num_repeat))
avgLength = sum(lengths)/num_repeat
errors.sort()
errorQuantile = errors[int(num_repeat*0.9)]
print("Average CI length = "+str(avgLength))
print("90% error is "+ str(errorQuantile))
print("CI length/90% error = "+str(avgLength/errorQuantile))

Results for Smooth based median, approximate DP
correct rate = 0.98
Average CI length = 1785.0882476563047
90% error is 1163.4743703094136
CI length/90% error = 1.5342738037122206


In [40]:
print(max(D))
print(count(D, 0, 178146))
print(counts[178146])

1490400.0
24421
24421.0


In [36]:
#To save time, pre-compute and store all counts


    
if mode==0:
    rad = 2*10000000
    counts = np.zeros(rad+1)
    counts[int(max(D)):rad+1]=n
    temp_count=1
    for i in range(n-1):
        if D[i+1]==D[i]:
            temp_count+=1
        else:
            for j in range(int(D[i]), int(D[i+1])):
                counts[j+int(rad/2)]=temp_count
            temp_count+=1
    
if mode==1:
    rad = 10000000
    counts = np.zeros(rad+1)
    counts[int(max(D)):rad+1]=n
    temp_count=1
    for i in range(n-1):
        if D[i+1]==D[i]:
            temp_count+=1
        else:
            for j in range(int(D[i]), int(D[i+1])):
                counts[j]=temp_count
            temp_count+=1


In [90]:
eps = [0.125, 0.25, 0.5, 1,2,4]
beta = [0.01, 0.025, 0.05, 0.1, 0.2]
len_eps = []
err_eps = []
for i in range(4,6):
    print("Results for SVT based median, Eps = "+str(eps[i]))
    errors = []
    lengths = []
    correct_count = 0
    num_repeat=1
    for j in range(num_repeat):
        Q,xl,xr,indi = SVTMedianCI(eps[i], beta[3], D)
        correct_count+=indi
        errors.append(abs(Q-D[int(n/2)]))
        lengths.append((xr-xl)/2)

    print("correct rate = " +str(correct_count/num_repeat))
    avgLength = sum(lengths)/num_repeat
    errors.sort()
    errorQuantile = errors[int(num_repeat*(1-beta[3]))]
    print("Average CI length = "+str(avgLength))
    print("error quantile is "+ str(errorQuantile))
    print("CI length/ error quantile = "+str(avgLength/errorQuantile))
    len_eps.append(avgLength)
    err_eps.append(errorQuantile)
    #print(errors)
print(len_eps)
print(err_eps)

Results for SVT based median, Eps = 2
correct rate = 1.0
Average CI length = 599.5
error quantile is 30.0
CI length/ error quantile = 19.983333333333334
Results for SVT based median, Eps = 4
correct rate = 1.0
Average CI length = 310.5
error quantile is 7.0
CI length/ error quantile = 44.357142857142854
[599.5, 310.5]
[30.0, 7.0]


In [96]:
eps = [0.125, 0.25, 0.5, 1,2,4]
beta = [0.01, 0.025, 0.05, 0.1, 0.2]
len_beta = []
err_beta = []
for i in range(2,4):
    print("Results for SVT based median, beta = "+str(beta[i]))
    errors = []
    lengths = []
    correct_count = 0
    num_repeat=100
    for j in range(num_repeat):
        Q,xl,xr,indi = SVTMedianCI(eps[3], beta[i], D)
        correct_count+=indi
        errors.append(abs(Q-D[int(n/2)]))
        lengths.append((xr-xl)/2)

    print("correct rate = " +str(correct_count/num_repeat))
    avgLength = sum(lengths)/num_repeat
    errors.sort()
    errorQuantile = errors[int(num_repeat*(1-beta[i]))]
    print("Average CI length = "+str(avgLength))
    print("error quantile is "+ str(errorQuantile))
    print("CI length/ error quantile = "+str(avgLength/errorQuantile))
    len_beta.append(avgLength)
    err_beta.append(errorQuantile)
print(len_beta)
print(err_beta)

Results for SVT based median, beta = 0.05
correct rate = 1.0
Average CI length = 1267.565
error quantile is 102.0
CI length/ error quantile = 12.427107843137255
Results for SVT based median, beta = 0.1
correct rate = 1.0
Average CI length = 1247.35
error quantile is 81.0
CI length/ error quantile = 15.39938271604938
[1267.565, 1247.35]
[102.0, 81.0]
